In [1]:
import tensorflow as tf

# 计算图
Tensorflow 中使用计算图来表示计算过程中各个计算操作的依赖关系，在tensorflow编程中，要首先定义计算图，然后再在tensorflow会话（session）中去运行计算图。

![dataflow graph](./images/tensors_flowing.gif "dataflow")

计算图有一下几个好处：
* **并行**： 由于计算图明确的表示了操作之间的关系，这为框架底层的优化算法提供了方便，使得计算更加容易被并行执行。
* **分布式执行**： 由于清晰的定义了数据在不同操作之间的流动关系，这方便了框架将数据分布到不同的设备、机器上去，进行分布式计算。
* **编译性**： tensorflow的编译器能够利用计算图生成更加高效的C++代码。
* **可移植性**：计算图是与编程语言无关的。比如，用户可以使用Python编写好计算图，再用C++导入。

tf.Graph 中包含了两部分的信息：
* **图结构**：包括图中的所有节点和边，它描述了一个计算图是如何由多个操作组成的
* **图集合**：包含图的元信息

In [2]:
# 分别定义了两个计算图，并分别定义了一个相同的变量“v”
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable("v", initializer=tf.zeros_initializer()(shape=[1]))

g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable("v", initializer=tf.ones_initializer()(shape=[1]))

with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))
    print(v)

with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))
    print(v)


[ 0.]
<tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
[ 1.]
<tf.Variable 'v:0' shape=(1,) dtype=float32_ref>


In [3]:
g3 = tf.Graph()
with g3.as_default():
    w1 = tf.Variable(tf.random_normal([2,3], stddev=1))
    w2 = tf.Variable(tf.random_normal([3,2], stddev=1))

    x = tf.placeholder(tf.float32, shape=(1,2), name="input")
    a = tf.matmul(x, w1)
    y = tf.matmul(a, w2)

with tf.Session(graph=g3) as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)

## 命名空间

tf.Graph 对象会为其包含的 tf.Operation 对象定义一个命名空间。TensorFlow 会自动为您的图中的每个指令选择一个唯一名称，但您也可以指定描述性名称，使您的程序阅读和调试起来更轻松。

TensorFlow API 提供两种方orFlow API 提供两种方法来覆盖操作名称：

* 创建新的tf.Operation接受一个 name 参数，TensorFlow会使用该参数为Tensor对象命名，如果名称已经存在，TensorFlow则会在附加上 "_1", "_2" 等字符

* 使用 tf.name_scope 函数，可以向在特定上下文中创建的所有操作添加名称作用域前缀。当前名称作用域前缀是一个用 "/" 分隔的名称列表，其中包含所有活动 tf.name_scope 上下文管理器的名称。如果某个名称作用域已在当前上下文中被占用，TensorFlow 将在该作用域上附加 "_1"、"_2" 等字符。例如：

In [5]:
g4 = tf.Graph()
# g4.reset_graph()
with g4.as_default():
    c_0 = tf.constant(0, name="c")
    c_1 = tf.constant(1, name="c")
    
    with tf.name_scope("outer"):
        c_2 = tf.constant(2, name="c")
    
        with tf.name_scope("inner"):
            c_3 = tf.constant(3, name="c")
    
        c_4 = tf.constant(4, name="c")
    
        with tf.name_scope("inner"):
            c_5 = tf.constant(5, name="c")

print(c_0)
print(c_1)
print(c_2)
print(c_3)
print(c_4)
print(c_5)

Tensor("c:0", shape=(), dtype=int32)
Tensor("c_1:0", shape=(), dtype=int32)
Tensor("outer/c:0", shape=(), dtype=int32)
Tensor("outer/inner/c:0", shape=(), dtype=int32)
Tensor("outer/c_1:0", shape=(), dtype=int32)
Tensor("outer/inner_1/c:0", shape=(), dtype=int32)


tf.Tensor 对象以生成输出张量的 tf.Operation 明确命名。张量名称的形式为 "OP_NAME:i"，其中：

* "OP_NAME" 是生成该张量的操作的名称。
* "i" 是一个整数，表示该张量在操作的输出中的索引。

## 将操作放置到不同的设备上

TensorFlow支持将程序跑在多个设备上，通过使用 tf.device 函数可以将特定上下文中创建的所有操作放置到同一设备。

**设备规范**具有以下形式：

/job:&lt;JOB_NAME&gt;/task:&lt;TASK_INDEX&gt;/device:&lt;DEVICE_TYPE&gt;:&lt;DEVICE_INDEX&gt;


* &lt;JOB_NAME&gt; 是一个字母数字字符串，并且不以数字开头。
* &lt;DEVICE_TYPE&gt; 是一种注册设备类型（例如 GPU 或 CPU）。
* &lt;TASK_INDEX&gt; 是一个非负整数，表示名为 &lt;JOB_NAME&gt; 的作业中的任务的索引。请参阅 tf.train.ClusterSpec 了解作业和任务的说明。
* &lt;DEVICE_INDEX&gt; 是一个非负整数，表示设备索引，例如用于区分同一进程中使用的不同 GPU 设备。


# 会话 tf.Session

TensorFlow 使用 tf.Session 类来表示客户端程序（通常为 Python 程序，但也提供了其他语言的类似接口）与 C++ 运行时之间的连接。tf.Session 对象使我们能够访问本地机器中的设备和使用分布式 TensorFlow 运行时的远程设备。它还可缓存关于 tf.Graph 的信息，使您能够多次高效地运行同一计算。

In [6]:
# 创建本地session
with tf.Session() as sess:
    pass

# 创建一个远程session
# with tf.Session("grpc://remote.org:2222"):
